# Day 2 - Lab 1: AI-Generated System Design & Database Seeding

**Objective:** Use the PRD artifact from Day 1 to generate a detailed SQL database schema, create realistic seed data, and then use those outputs to create and seed a live, local database file.

**Estimated Time:** 150 minutes

**Introduction:**
Welcome to Day 2! Today, we transition from *what* we're building to *how* we'll build it. In this lab, you will act as the lead architect for the Onboarding Tool. Your task is to use the PRD to define the data structure of the application and create a tangible database artifact that will be used for the rest of the course.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

We will load the `day1_prd.md` artifact from Day 1. This document is the primary source of truth for our project and provides the necessary context for the LLM to make intelligent design suggestions.

**Model Selection:**
Feel free to experiment with different models by changing the `model_name` in `setup_llm_client()`. Models with strong reasoning capabilities, like `gpt-4o`, `o3`, or `gemini-2.5-pro`, are excellent choices for design tasks.

**Helper Functions Used:**
- `setup_llm_client()`: To configure the API client.
- `get_completion()`: To send prompts to the LLM.
- `load_artifact()`: To read the PRD from the `artifacts` directory.
- `save_artifact()`: To save the generated SQL schema and seed data.
- `clean_llm_output()`: To remove markdown fences from the generated SQL code.

In [2]:
import sys
import os
import sqlite3

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, clean_llm_output

client, model_name, api_provider = setup_llm_client(model_name="gpt-4o")

# Load the PRD from Day 1
prd_content = load_artifact("day1_prd.md")
if not prd_content:
    print("Warning: Could not load day1_prd.md. Lab may not function correctly.")

✅ LLM Client configured: Using 'openai' with model 'gpt-4o'


## Step 2: The Challenges

### Challenge 1 (Foundational): Generating the SQL Schema

**Task:** Use the PRD to generate a normalized SQL schema for the application.

**Instructions:**
1.  Create a prompt that instructs the LLM to act as a Database Administrator (DBA).
2.  Provide the `prd_content` as context.
3.  Ask the LLM to design a normalized SQL schema with at least two tables (e.g., `users` and `onboarding_tasks`).
4.  The output should be the raw `CREATE TABLE` statements.
5.  Save the generated SQL to `artifacts/schema.sql`.

In [5]:
# TODO: Write a prompt to generate the SQL schema from the PRD.
schema_prompt = f"""
# Pretend you are a Database Administrator. Based on the following Product Requirements Document (PRD), 
# generate a SQL schema to support the application's data needs. 
# Ensure that the schema includes tables, columns, data types, primary keys, 
# foreign keys, and any necessary relationships between tables.
Make the structure simple but robust, avoiding unnecessary complexity.
Make sure the bare minimum is met, nothing fancy
It should be in SQlite dialect.
Here is the PRD:
{prd_content}
Please give me the raw CREATE TABLE statements
"""

print("--- Generating SQL Schema ---")
if prd_content:
    generated_schema = get_completion(schema_prompt, client, model_name, api_provider)
    
    # Clean up the generated schema using our helper function
    cleaned_schema = clean_llm_output(generated_schema, language='sql')
    print(cleaned_schema)
    
    # Save the cleaned schema
    save_artifact(cleaned_schema, 'schema.sql')
else:
    print("Skipping schema generation because PRD is missing.")
    cleaned_schema = ""

--- Generating SQL Schema ---
CREATE TABLE Users (
    user_id INTEGER PRIMARY KEY AUTOINCREMENT,
    username TEXT NOT NULL UNIQUE,
    email TEXT NOT NULL UNIQUE,
    password TEXT NOT NULL,  -- In a real application, passwords should be hashed
    created_at DATETIME DEFAULT CURRENT_TIMESTAMP
);

CREATE TABLE Movies (
    movie_id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    release_date DATE,
    synopsis TEXT,
    duration INTEGER,  -- Duration in minutes
    metadata BLOB -- Additional metadata like cast, crew, etc.
);

CREATE TABLE TV_Shows (
    show_id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    release_date DATE,
    synopsis TEXT,
    seasons INTEGER,
    episodes INTEGER,
    metadata BLOB -- Additional metadata like cast, crew, etc.
);

CREATE TABLE User_Watchlist (
    watchlist_id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER NOT NULL,
    movie_id INTEGER,
    show_id INTEGER,
    status TEXT CHECK(status IN ('planned', 'w

### Challenge 2 (Intermediate): Generating Realistic Seed Data

**Task:** Prompt the LLM to generate realistic seed data that conforms to the schema you just created.

**Instructions:**
1.  Create a new prompt.
2.  Provide both the `prd_content` and the `cleaned_schema` as context.
3.  Instruct the LLM to generate 5-10 realistic `INSERT` statements for your tables.
4.  The data should be relevant to a new hire onboarding tool (e.g., sample user names and task titles like "Complete HR Paperwork").
5.  Save the generated `INSERT` statements to `artifacts/seed_data.sql`.

In [12]:
# TODO: Write a prompt to generate realistic seed data.
seed_data_prompt = f"""
# Generate realistic seed data for the following SQL schema. Needs to be SQLlite
# Ensure that the data types match the schema and that the data is coherent and plausible.
Make sure that all problematic characters are escaped properly.
Do 10 rows for each table
Here is the SQL schema:
{cleaned_schema}
And here is the content of the PRD to help you understand the context:
{prd_content}
Please provide INSERT statements to populate the tables with sample data.   
"""

print("--- Generating Seed Data ---")
if prd_content and cleaned_schema:
    generated_seed_data = get_completion(seed_data_prompt, client, model_name, api_provider)
    
    # Clean up the generated seed data
    cleaned_seed_data = clean_llm_output(generated_seed_data, language='sql')
    print(cleaned_seed_data)
    
    # Save the cleaned seed data
    save_artifact(cleaned_seed_data, 'seed_data.sql')
else:
    print("Skipping seed data generation because PRD or schema is missing.")

--- Generating Seed Data ---
-- Insert data into Users table
INSERT INTO Users (username, email, password) VALUES
('john_doe', 'john@example.com', 'password123'),
('jane_smith', 'jane@example.com', 'securepass456'),
('alex_brown', 'alex@example.com', 'mypassword789'),
('luke_white', 'luke@example.com', 'passw0rd'),
('emily_green', 'emily@example.com', 'emilypass987'),
('chris_blue', 'chris@example.com', 'bluepass654'),
('kate_black', 'kate@example.com', 'blackpass321'),
('mike_red', 'mike@example.com', 'redpass111'),
('sarah_yellow', 'sarah@example.com', 'yellowpass222'),
('david_gray', 'david@example.com', 'graypass333');

-- Insert data into Movies table
INSERT INTO Movies (title, release_date, synopsis, duration, metadata) VALUES
('The Great Adventure', '2022-01-15', 'A thrilling adventure of a lifetime.', 120, x''),
('Romantic Getaway', '2021-05-22', 'A romantic story set in the beautiful Paris.', 100, x''),
('Sci-Fi Future', '2023-03-10', 'A glimpse into the future with advanced t

### Challenge 3 (Advanced): Creating and Seeding a Live Database

**Task:** This is a critical technical step. You will write a Python script to execute the generated SQL files, creating a live `onboarding.db` file that your application will use.

**Instructions:**
1.  Complete the `create_database` function below.
2.  The function should first connect to (and thus create) a SQLite database file named `artifacts/onboarding.db`.
3.  It should then open and execute the `schema.sql` file to create the tables.
4.  Finally, it should open and execute the `seed_data.sql` file to populate the tables.
5.  Use a `try...finally` block to ensure the database connection is always closed, even if an error occurs.

> **Hint:** The `try...finally` block is a crucial Python pattern. The code in the `finally` block will run whether the `try` block succeeds or fails, making it the perfect place to ensure resources like database connections are always closed.

**Expected Quality:** A physical `onboarding.db` file in your `artifacts` folder. This is a tangible asset that proves your design is valid and provides a concrete foundation for backend development.

In [3]:
def create_database(db_path, schema_path, seed_path):
    """Creates and seeds a SQLite database from SQL files."""
    if not os.path.exists(schema_path):
        print(os.getcwd())
        print(f"Error: Schema file not found at {schema_path}")
        return
    
    conn = None
    try:

        # Delete onboarding.db if it exists to start fresh each time
        if os.path.exists(db_path):
            os.remove(db_path)
            print(f"Existing database at {db_path} removed to start fresh.")

        # TODO: Connect to the SQLite database. This will create the file if it doesn't exist.
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        print(f"Successfully connected to database at {db_path}")

        # TODO: Read the content of the schema file using load_artifact.
        schema_sql = ""
        schema_sql = load_artifact(schema_path)
        if not schema_sql:
            print(f"Error: Could not read schema file at {schema_path}")
            return 
        
        # TODO: Execute the schema SQL script.
        # Hint: Use cursor.executescript() for multi-statement SQL strings.
        # Make the script only execute if the db is not empty though (so no double importingg
        cursor.executescript(schema_sql)    
        
        print("Tables created successfully.")
        


        # TODO: Check if the seed data file exists. If it does, load and execute it.
        if os.path.exists(seed_path):
            # Load and execute the seed data
            seed_sql = load_artifact(seed_path)

            cursor.executescript(seed_sql)
            print("Database seeded successfully.")


        # TODO: Commit the changes to the database.
        conn.commit()
        print("Database changes committed.")        
        
    except sqlite3.Error as e:
        print(f"Database error: {e}")
    finally:
        # TODO: Ensure the connection is closed if it was opened.
        if conn:
            conn.close()
            print("Database connection closed.")
# Define file paths
print(os.getcwd())
db_file = os.path.join(project_root, "artifacts", "my_cinema.db")
schema_file = os.path.join(project_root, "artifacts", "schema.sql")
seed_file = os.path.join(project_root, "artifacts", "seed_data.sql")

# Execute the function
create_database(db_file, schema_file, seed_file)

# Generate a show database method so I can see the tables and data
def show_database_contents(db_path):
    """Displays the tables and their contents in the SQLite database."""
    if not os.path.exists(db_path):
        print(f"Error: Database file not found at {db_path}")
        return
    
    conn = None
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        print(f"Connected to database at {db_path}")

        # Fetch all table names
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()
        
        if not tables:
            print("No tables found in the database.")
            return
        
        for table_name in tables:
            table_name = table_name[0]
            print(f"\nContents of table '{table_name}':")
            cursor.execute(f"SELECT * FROM {table_name};")
            rows = cursor.fetchall()
            
            if not rows:
                print("  (No data)")
            else:
                for row in rows:
                    print("  ", row)
                    
    except sqlite3.Error as e:
        print(f"Database error: {e}")
    finally:
        if conn:
            conn.close()
            print("Database connection closed.")

show_database_contents(db_file)

c:\Users\labadmin\Documents\AG-AISOFTDEV (rename)\Labs\Day_02_Design_and_Architecture
Successfully connected to database at c:\Users\labadmin\Documents\AG-AISOFTDEV (rename)\artifacts\my_cinema.db
Tables created successfully.
Database seeded successfully.
Database changes committed.
Database connection closed.
Connected to database at c:\Users\labadmin\Documents\AG-AISOFTDEV (rename)\artifacts\my_cinema.db

Contents of table 'Users':
   (1, 'john_doe', 'john@example.com', 'password123', '2025-09-05 17:51:32')
   (2, 'jane_smith', 'jane@example.com', 'securepass456', '2025-09-05 17:51:32')
   (3, 'alex_brown', 'alex@example.com', 'mypassword789', '2025-09-05 17:51:32')
   (4, 'luke_white', 'luke@example.com', 'passw0rd', '2025-09-05 17:51:32')
   (5, 'emily_green', 'emily@example.com', 'emilypass987', '2025-09-05 17:51:32')
   (6, 'chris_blue', 'chris@example.com', 'bluepass654', '2025-09-05 17:51:32')
   (7, 'kate_black', 'kate@example.com', 'blackpass321', '2025-09-05 17:51:32')
   (8

## Lab Conclusion

Excellent work! You have now moved from abstract requirements to a concrete, physical database artifact. You've used an LLM to design a schema, generate realistic test data, and then used a Python script to bring that database to life. This `onboarding.db` file is the foundation upon which we will build our API in Day 3.

> **Key Takeaway:** The ability to generate structured data definitions (like a SQL schema) from unstructured text (like a PRD) is a core skill in AI-assisted development. It automates a critical and often time-consuming design step.